# Filling in the Gaps

You're probably still a little confused about a few things from the lesson:

* Why does `print(bytes([10, 100, 200])) -> b'\nd\xc8'` looks so odd? 
    * What's with the `\n`? 
    * What's with the `\x`? 
    * Are the `c` and the `8` related to the `\x`?
    * Is `d` related to either `\n`, `\x`, or neither?
* How should we interpret these exotic byte literals when you encounter one _in the wild_?
* How does the magical `int.from_bytes` function work?
* How does its inverse function `int.to_bytes` work?

To answer these questions we will make a `bites` class that acts just like `bytes`, but with a few minor differences: 
* Prints itself with strings (`"\x00\xff"`) instead of instead of the special `bytes` literals (`b"\x00\xff"`) .
* Will duplicate the functionality of the magical `int.from_bytes` and `int.to_bytes` methods by implementing its `bites.from_int` and `bites.to_int` methods.

Here's a skeleton of the class:

In [ ]:
class bites:
    
    def __init__(self, ???):
        ???
        
    def __repr__(self):
        return "(cheating) " + repr(self.values)

    def to_int(self, byte_order):
        raise NotImplementedError()
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        raise NotImplementedError()

# Representation & Numbers

A "byte" is a value `x` such that `0 <= x < 256`.

Simple.

How can we represent a number above 256 as a series of bytes? Hmm ...

Let's think about how we solve this with our normal decimal numbers. Instead of 256 separate values or "places", we only have 10 places with values `y` such that `0 <= y < 9`.

What happens when we encounter a number outside this range? Say the number after 9, `9 + 1`? We no longer have a "numberal" or number symbol to represent it. We could invent one of course, but that would be a memorization nightmare ...

Instead we employ an ingenius trick to utilize _two numerals to rempresent 1 number_. It's really clever when you think of it. Of course, we write this `9 + 1` as `10` -- the left character represents that we have run out of numerals 1 time. It's the 10's place. The right numberal says that we haven't started going up again. We can repeat this procedure indefinitely. Here's the number 518:

<center>

```
     0              5           1         8
----------  ...  ---------- ---------- ----------
  10**n's          100's       10's       1's  

``` 
</center>

Therefore, any number `N` can be represented as a sum


$$ \sum_{i=0}^{i=n} a_{i} *  10^i = a_n *  10^n + ... + a_2 *  10^2 + a_1 *  10^1 + a_{0} *  10^0 $$


We can employ exactly the same trick to represent numbers greater than 256 as a series of bytes (numbers 0 <= x < 256). Just replaces the old threshold 10 with the new threshold 256:

$$ \sum_{i=0}^{i=n} a_{i} * 256^i =  a_n * 256^n + ... + a_2 * 256^2 + a_1 * 256^1 + a_{0} * 256^0 $$

The trick here is to go from an `int` to a list of values where each item represents a place in the base 256 expansion of the `int` we're dealing with. Here's the number 518 (2 * 256^1 + 6 * 256^0):

<center>
    
```
   0                0           2         6
----------  ...  ---------- ---------- ----------
 256**n's          65536's     256's       1's  

```
</center>

So we have a representation. In Python we could use the `list` data structure to store this: $[a_n, ..., a_2, a_1, a_0]$. 

The decimal representation of 518 would have $a_2 = 5$, $a_1 = 1$, $a_0 = 8$ and leaving us with a representation of `[5, 1, 8]`. 

The base 256 respresentation of 518 would have $a_1 = 2$ and $a_0 = 6$ leaving us with a representation of `[2, 6]`

Let's update the `bites` constructor to use this representation:

In [ ]:
class bites:
    
    def __init__(self, values):
        # a list of numbers x: 0 <= x < 256
        self.values = values
        
    def __repr__(self):
        return "(cheating) " + repr(self.values)

    def to_int(self, byte_order):
        raise NotImplementedError()
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        raise NotImplementedError()

Now we just need an algorithm to go back and forth between `bites` and `int`. Let's start with the `bites` -> `int` direction.

# Implement `bites.to_int(places)`

Here's a number we will be working with throughout this lesson:

In [ ]:
N = 92837365

Here is the list representation of the decimals / base-10 expansion of this number as discussed above:

In [ ]:
PLACES = [9, 2, 8, 3, 7, 3, 6, 5]

## Step 1: `decimal_places_to_int(places)`

Your first exercise is to write a `decimal_places_to_int(places)` function. It's correct if `decimal_places_to_int(PLACES) == N`.

This is a tricky exercise so take your time with it!

In [ ]:
def decimal_places_to_int(places):
    raise NotImplementedError()

In [ ]:
def test():
    assert decimal_places_to_int(PLACES) == N
    print("Test passed!")

test()

([answers.py](./answers.py) if you can't figure it out)


## Step 2: `base_256_places_to_int(places)`

Now do it with base-256.

It's correct if `base_256_places_to_int([5, 136, 149, 245]) == N`

This one's quite a bit easier!

In [ ]:
def base_256_places_to_int(places):
    raise NotImplementedError()

In [ ]:
def test():
    assert base_256_places_to_int([5, 136, 149, 245]) == N
    print("Test passed!")

test()

## Step 3: `places_to_int(places, base)`

Modify `base_256_places_to_int` so that it accepts arbitrary bases (e.g. 10 or 256):

In [ ]:
def places_to_int(places, base):
    raise NotImplementedError()

In [ ]:
def test():
    assert places_to_int([5, 136, 149, 245], 256) == N
    assert places_to_int([int(x, 16) for x in hex(N)[2:]], 16) == N
    assert places_to_int([int(x, 2) for x in bin(N)[2:]], 2) == N
    print("Test passed!")

test()

## Step 4: `places_to_int(places, base, byte_order)`

This is how we've been choosing to represent the number `92837365` in base-256: `[5, 136, 149, 245]`

The larger places on the left, smaller places to the right. Just like familiar decimal nubmers

```
    5     136     149     245
-----   -----   -----   -----
256^3   256^2   256^1   256^0
```

But isn't this choice completely arbitrary? Why not do the opposite: smaller places to the left, larger places to the right? 

```
  245     149     136       5
-----   -----   -----   -----
256^0   256^1   256^2   256^3
```

These choices are completely arbitrary! Different areas of computer science prefer one way or the other.

The order we've been using up to this point -- big places to the left, little places to the right -- is called **"Big Endian"**. This is preferred in network programming.

The second one -- little places to the left, big places to the right -- is called **"Little Endian"**. This is how computers usually store information internally.

Even within Bitcoin, Satoshi didn't choose one way or the other. He generally preferred "little endian" but he encodes IP addresses using "big endian". What a mess!

You can find an interesting discussion of "endianness" [here](https://bitcoin.stackexchange.com/questions/2063/why-does-the-bitcoin-protocol-use-the-little-endian-notation) and a nice YouTube video [here](https://www.youtube.com/watch?v=seZLUbgbB7Y)


##### Exercise: add another parameter to our function so that it can handle "Little Endian" byte order. 

* If `byte_order` is `"little"` you just need to reverse the list:
* Raise a `ValueError` if `byte_order` isn't `"big"` or `"little"`

In [ ]:
from utils import assert_raises

def places_to_int(places, base, byte_order):
    raise NotImplementedError()

In [ ]:
def test():
    assert places_to_int([5, 136, 149, 245], 256, 'big') == N
    assert places_to_int([5, 136, 149, 245][::-1], 256, 'little') == N
    
    assert places_to_int([int(x, 16) for x in hex(N)[2:]], 16, 'big') == N
    assert places_to_int([int(x, 16) for x in hex(N)[2:]][::-1], 16, 'little') == N

    assert_raises(places_to_int, [], 10, 'foo')
    
    print("Test passes")
    
test()

## Step 5: `bites.to_int(base, byte_order)`

In [ ]:
class bites:
    
    def __init__(self, values):
        self.values = values
        
    def __repr__(self):
        return "(cheating) " + repr(self.values)
    
    def to_int(self, byte_order):
        raise NotImplementedError()
        
    @classmethod
    def from_int(cls, n, length, byte_order):
        raise NotImplementedError()

In [ ]:
def test():
    places = [5, 136, 149, 245]
    assert bites(places).to_int('big') == N
    assert bites(places[::-1]).to_int('little') == N

test()

You've now completely implemented the functionality provided by `int.from_bytes`. Nice work! Now to the next method ...

# Implement `bites.from_int(...)`

`bites.to_int` handled `bites`-to-`int` conversions. Now let's implement a `bites.from_int` classmethod to do `int`-to-`bites` conversions. 


### `classmethod`

If you want a refresher on classmethods, check out [this video](https://www.youtube.com/watch?v=rq8cL2XMM5M&t=18s). They're useful when you want to create an instance of a class from data which doesn't fit in the class' `__init__` [constructor](https://stackoverflow.com/a/8986413/2542016). In our case, we can't call `bites(92837365)` because the bites constructor expects a `list` and not an `int`. We could put some fancy logic in our `bites.__init__` to check if the argument is an integer and convert if it is -- but that is frowned upon because it makes the `__init__` function hard to read. 

Instead, we will write a `bites.from_int` classmethod which will allow us to create `bites` instances directly from `int`s: 

> ```some_bites = bites.from_int(some_int)```

It will translate the `int` into a `list` and then pass that to the constructor `bites.__init__`. Our constructor stays nice and clean and we can still do type conversions!

### Division

`bytes.to_int` used multiplication: we started with a list of values, multified them by coefficients according to some rules, and added up the result.

`bytes.from_int` will use division. We will start with a number and divide off the base-256 places one at a time. It will be like dealing from a deck of cards ....

![ChessUrl](https://tenor.com/view/ref-with-yellow-cards-fifa18-pose-throw-cards-gif-12023802.gif "chess")


Division can be confusing. [Watch the next 10 seconds of this childrens' video for a refresher](https://youtu.be/KGMf314LUc0?t=76).

To compute $\frac{x}{y}$, we just just take groups of `y` from `x` until some number less than `y` is left. What's left is called the "remainder". 

Here's some notation and vocab review to jog your memory:

$$
\require{enclose}
\begin{array}{rll}
    quotient && \hbox{+ remainder} \\[-3pt]
   divisor \enclose{longdiv}{dividend}\kern-.2ex \\[-3pt]
  \end{array}
$$

$$
\frac{dividend}{divisor} = quotient + \frac{remainder}{divisor} \hspace{1cm} \hbox{where remainder is < divisor} 
$$

### Division in Python

Python has 3 primary division operators:

`/` is floating point division

`//` is floor division

`%` is the "modulus" (remainder after division)

In [ ]:
# Floating point division
# Returns quotient + remainder / divisor as a float

5 / 2

In [ ]:
# Floor division 
# Returns the quotient, discards the remainder

5 // 2

In [ ]:
# Modulus
# Returns the remainder

5 % 2

Let's attempt to use these division operators to deconstruct the hex representation of N:

In [ ]:
hex(N)

In [ ]:
# Modulus gives us the right-most place
hex(N % 16)

In [ ]:
# Floor division gives us the rest of the places
hex(N // 16)

In [ ]:
# Combining // and % can traverse the hex representation
# This computs the 2nd place

hex(N // 16 % 16)

In [ ]:
# How they're related
N == 16 * (N // 16) + (N % 16)

Now you know how to compute the 1's place ($16^0$) of the hexidecimal representation of N.

##### Exercise: Compute the 1's place ($256^0$) of the base-256 representation of N

In [ ]:
def get_ones_place():
    return N % 256

In [ ]:
def test_get_ones_place():
    assert get_ones_place() == 245
    print("Test passed")

test_get_ones_place()

Now you know how to compute the 1's place ($16^0$) of the hexidecimal and base-256 representations of N.

##### Exercise: Compute the 65536's place ($256^2$) using only the `//` and `%` operators?

In [ ]:
def get_65536s_place():
    raise NotImplementedError()

In [ ]:
def test_get_65536s_place():
    assert get_65536s_place() == 136
    print("Test passed")
    
test_get_65536s_place()

## Step 1:  `int_to_256_places(n)`

Use the techniques above to compute base-256 representation of numbers of the form discussed in `bites.to_int` section above.

If you can't get it, check out the table 3 cells down ....

In [ ]:
def int_to_base_256_places(n):
    raise NotImplementedError()

In [ ]:
def test():
    assert int_to_base_256_places(N) == [5, 136, 149, 245]
    print("Test passed")
    
test()

Here's how my solution works:

**step**|**n**|**places**|**explanation**
:-----:|:-----:|:-----:|:-----:
 |92837365|[]|initial
%|92837365|[245]|92837365 % 256 == 245
//|362645|[245]|92837365 // 256 == 362645
%|362645|[149, 245]|362645 % 256 == 149 ; insert at index 0
//|1416|[149,  245]|362645 // 256 == 1416
%|1416|[136, 149, 245]|1416 % 256 == 136 ; insert at index 0
//|5|[136, 149, 245]|1416 // 256 == 5
%|5|[5, 136, 149, 245]|5 % 256 == 5 ; insert at index 0
//|0|[5, 136, 149, 245]|5 // 256 == 0
...|...|...|loop terminates

See how it's like dealing cards? With every loop we grab off the modulus and wipe out 1 factor of 256 from the polynomial expansion of `n` so that the next time we look for the modulus we'll get the next place. (FIXME: formatting below sucks)

$$ a_n * 256^n + ... + a_1 * 256^1 + a_{0} * 256^0  \mod 256 = a_0 $$
$$ a_n * 256^n + ... + a_1 * 256^1 + a_{0} * 256^0  // 256 = a_{n} * 256^{n-1} + ... + a_1 * 256^0 + 0  $$
$$  a_{n-1} * 256^{n-1} + ... + a_1 * 256^0  \mod 256 = a_1 $$
$$ ... $$

## Step 2: `int_to_places(n, base)`

Now add a second parameter to the function and rename it to `int_to_places(n, base)` so that it works with any base (e.g. 10 or 256).

Test it against the binary, octal, and hex representations we say above. For example, we saw `hex(92837365)` was `0x58895f5`, so as a list it should be `[5, 8, 8, 9, 5, 15, 5]` (hex `f` equals decimal `15`).

In [ ]:
def int_to_places(n, base):
    raise NotImplementedError()

print(int_to_places(N, 16))  # is this what you expect?

In [ ]:
def test():
    assert int_to_places(N, 256) == [5, 136, 149, 245]
    assert int_to_places(N, 16) == [int(x, 16) for x in hex(N)[2:]]
    assert int_to_places(N, 2) == [int(x, 2) for x in bin(N)[2:]]
    print("Test passed")

test()

In [ ]:
(256).to_bytes(1, 'big')

## Step 3: `int_to_places(n, base, length)`

Remember how every field in the protocol docs' "Message Structure" table has a `length` attribute?

![image](../images/message-structure.png)

We need to be able to support that, too. We should be able to say `int_to_places(1, 4)` and get [0, 0, 0, 1]. This feature helps us interpret and produce n-byte integer fields we encounter in the Bitcoin protocol.

Raise a `ValueError` if `n` doesn't fit in that many `bites` using the given `base`

In [ ]:
0 // 256, 0 % 256

In [ ]:
def int_to_places(n, base, length):
    raise NotImplementedError()

In [ ]:
def test():
    assert int_to_places(N, 256, 10) == [0] * 6 + [5, 136, 149, 245]
    
    vals = [int(x, 16) for x in hex(N)[2:]]
    zeros = [0] * (20 - len(vals))
    assert int_to_places(N, 16, 20) == zeros + vals

    # You need 2 places to encode 16 in hex (0x10)
    assert_raises(int_to_places, 16, 16, 1)

test()

## Step 4:  `int_to_places(n, base, length, byte_order)`

Add another parameter to our function so that it can handle "Little Endian" byte order. If the `byte_order` isn't `"little"` or `"big"` then raise a `ValueError`.

In [ ]:
def int_to_places(n, base, length, byte_order):
    raise NotImplementedError()

In [ ]:
def test():
    vals = [5, 136, 149, 245]
    assert int_to_places(N, 256, len(vals), 'big') == vals
    assert int_to_places(N, 256, len(vals), 'little') == vals[::-1]

    vals = [int(x, 16) for x in hex(N)[2:]]
    assert int_to_places(N, 16, len(vals), 'big') == vals
    assert int_to_places(N, 16, len(vals), 'little') == vals[::-1]

    assert_raises(int_to_places, 1, 10, 1, 'dog')
    
    print("Test passed")

test()

## Step 5:  `bites.from_int(n, base, length, byte_order)`


Let's put it all together. Fill out the `from_int` method below and get the tests to pass

In [ ]:
class bites:
    
    def __init__(self, values):
        self.values = values
        
    def __repr__(self):
        return "(cheating) " + repr(self.values)
    
    def to_int(self, byte_order):
        return places_to_int(self.values, 256, byte_order)
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        raise NotImplementedError()

In [ ]:
def test():
    vals = [5, 136, 149, 245]
    assert bites.from_int(N, len(vals), 'big').values == vals
    assert bites.from_int(N, len(vals), 'little').values == vals[::-1]
    
    # round trip
    assert bites.from_int(N, 4, 'big').to_int('big') == N
    
    print("Test passed")

test()

# `bites.__repr__`

The representations of Python objects are determined by `.__repr__()` methods.

Let's see `bytes.__repr__` in action:

In [ ]:
for i in range(256):
    print(i, "->", bytes([i]))

I want you to implement a function that can print `bites` instances in the same way. To assist with this one I'm going to give you a list of character codes that have special meaning to `bytes`.

Below is a dictionary containing an `int -> ascii character` mapping of all numbers in 0 <= x < 256 with special meaning to `bytes`.

In [ ]:
from utils import special_chars

print(special_chars)

Any value left unassigned by that dictionary should be converted into "\x" + hexidecimal representation. For example the number 150 is ontside the dictionary. It would therefore be presented as `"\x96"` because it's hexidecimal representation is `96`.

### Exercise: Implement a  `represent` function that works exactly like `bytes.__repr__` but with strings

Here's how it should work:

`represent(bytes([145, 22, 75, 1, 83])) -> "\\x91\\x16K\\x98S"`

We need to escape the `\` with another `\` because characters following `\x` will be converted to Unicode by python's string engine:

In [ ]:
'\xff'  # We don't want this!!

In [ ]:
def represent(b):
    raise NotImplementedError()

# How does it look?
represent(bites([145, 22, 75, 1, 83]))

In [ ]:
def test():
    assert represent(bites([145, 22, 75, 1, 83])) == "\\x91\\x16K\\x01S"
    print("Test passed")
    
test()

# Put it all together (with some help from our friends)

I'm going to add 2 methods that our Lesson 1 code requires: `.strip` and `__eq__`. To simplify things I just convert to `bytes` and have it do all the work. I'll explain these after the exercise ...

Go ahead an copy your solution from the last exercise into place as `bites.__repr__`:

In [ ]:
class bites:
    
    def __init__(self, values):
        self.values = values
    
    def __eq__(self, other):
        return self.values == other.values
    
    def __repr__(self):
        raise NotImplementedError()
        
    def to_int(self, byte_order):
        return places_to_int(self.values, 256, byte_order)
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        places = int_to_places(n, 256, length, byte_order)
        return cls(places)

    def strip(self):
        return bites(list(bytes(self.values).strip(b"\x00")))

# How does it look?
bites([145, 22, 75, 1, 83])

In [ ]:
def test():
    assert repr(bites([145, 22, 75, 152, 83])) == "\\x91\\x16K\\x98S"
    print("Test passed")
    
test()

### `bites.__eq__`

needed for  magic bytes comparisons

In [ ]:
bites([0xf9, 0xbe, 0xb4, 0xd9]) == bites([0xF9, 0xBE, 0xB4, 0xD9])

### `bites.strip()` needed for reading commands

In [ ]:
b = bites(list(b"version\x00\x00\x00\x00\x00"))

print("unstripped:", b)
print("stripped:", b.strip())

### `BiteStream`

This class turns streams of `bytes` into streams of `bites`

In [ ]:
class BitesStream:

    def __init__(self, stream):
        self.stream = stream

    def read(self, n):
        return bites(list(self.stream.read(n)))
        
    def __getattr__(self, name):
        return getattr(self.stream, name)

### Hashing `bites`

`hashlib.sha256` requires inputs to the "Buffer API" which we won't bother with (you've got to implement it in C ...)

In [ ]:
from hashlib import sha256

def compute_checksum(b):
    hashed = sha256(sha256(bytes(b.values)).digest()).digest()
    checksum = hashed[:4]
    return b.__class__(list(checksum))

# Reading Bitcoin Messages From `bites`

A couple small tweeks to make our `NetworkEnvelope` class developed in Lesson 1 work with `bites` instead of `bytes`

In [ ]:
NETWORK_MAGIC = bites([0xf9, 0xbe, 0xb4, 0xd9])

class NetworkEnvelope:

    def __init__(self, command, payload):
        self.command = command
        self.payload = payload

    @classmethod
    def from_stream(cls, stream):
        magic = stream.read(4)
        if magic != NETWORK_MAGIC:
            raise ValueError('Network magic is wrong')

        command = stream.read(12).strip()
        payload_length = stream.read(4).to_int('little')
        checksum = stream.read(4)
        payload = stream.read(payload_length)
        
        if checksum != compute_checksum(payload):
            raise RuntimeError("Checksums don't match")

        return cls(command, payload)

    def __repr__(self):
        return f"<NetworkEnvelope command={self.command}>"

In [ ]:
import socket

# magic "version" bytestring
VERSION = b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00j\x00\x00\x00\x9b"\x8b\x9e\x7f\x11\x01\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x93AU[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rV\xc5C\x9b:\xea\x89\x14/some-cool-software/\x01\x00\x00\x00\x01'

PEER_IP = "35.167.105.93"
PEER_PORT = 8333

sock = socket.socket()
sock.connect((PEER_IP, PEER_PORT))
stream = sock.makefile('rb')
bites_stream = BitesStream(stream)

# initiate the "version handshake"
sock.send(VERSION)

# receive their "version" response
msg = NetworkEnvelope.from_stream(bites_stream)

print(msg)
print(msg.payload)